In [ ]:
# https://towardsdatascience.com/bert-text-classification-in-3-lines-of-code-using-keras-264db7e7a358

In [2]:
!pip3 install ktrain

     |████████████████████████████████| 25.3MB 1.4MB/s 
     |████████████████████████████████| 6.8MB 43.1MB/s 
     |████████████████████████████████| 983kB 50.9MB/s 
     |████████████████████████████████| 266kB 50.8MB/s 
     |████████████████████████████████| 1.3MB 40.8MB/s 
     |████████████████████████████████| 1.2MB 48.9MB/s 
     |████████████████████████████████| 471kB 39.2MB/s 
     |████████████████████████████████| 2.9MB 40.6MB/s 
     |████████████████████████████████| 890kB 40.4MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.3-cp36-none-any.whl size=25276312 sha256=826f046690c79690f9fbf77744cb4fc5de5648775a33973ae793b6a83c7363c2
  Stored in directory: /root/.cache/pip/wheels/86/cd/9c/ad53b98a3eccf4ddeddccfce99d353781f206a1ac4287ca721
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993194 sha256=5bcd885847d1f073a60bd8db13c10d0b076149adf699373c761435b5b1973af2
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

In [3]:
# import ktrain
import ktrain
from ktrain import text

In [ ]:
ktrain.__version__

'0.25.3'

In [4]:
import tensorflow as tf
# dataset = tf.keras.utils.get_file(
#     fname="aclImdb.tar.gz", 
#     origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
#     extract=True,
# )

In [7]:
import os.path
dataset = '/content/drive/MyDrive/PY/SemEval2021'
IMDB_DATADIR = os.path.join(os.path.dirname(dataset), 'text_humor.csv')
print(IMDB_DATADIR)

/content/drive/MyDrive/PY/text_humor.csv


## STEP 1:  Load and Preprocess the Dataset

The `texts_from_folder` function will load the training and validation data from the specified folder and automatically preprocess it according to BERT's requirements.  In doing so, the BERT model and vocabulary will be automatically downloaded.

In [10]:
trn, val, preproc = text.texts_from_csv('/content/drive/MyDrive/PY/SemEval2021/data/train/text_humor.csv',
                    'text',['is_humor'],random_state=0,preprocess_mode='bert',
                    val_filepath='/content/drive/MyDrive/PY/SemEval2021/data/test/test_text_humor.csv')

detected encoding: utf-8 (if wrong, set manually)
preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [ ]:
trn, val, preproc = text.texts_from_folder(IMDB_DATADIR, 
                                          maxlen=50, 
                                          preprocess_mode='bert',
                                          train_test_names=['train', 
                                                            'test'],
                                          classes=['pos', 'neg'])

## STEP 2:  Load a pretrained BERT model and wrap it in a `ktrain.Learner` object

In [24]:
model = text.text_classifier('bert', trn, preproc=preproc)
learner = ktrain.get_learner(model,train_data=trn, val_data=val, batch_size=6)

Is Multi-Label? False
maxlen is 400
done.


## STEP 3:  Train and Fine-Tune the Model on the IMDb dataset

We employ the `learner.fit_onecycle` method in *ktrain* that employs the use of a [1cycle learning  rate schedule](https://arxiv.org/pdf/1803.09820.pdf).  We use a learning rate of 2e-5 based on recommendations from [the original paper](https://arxiv.org/abs/1810.04805).

As can be seen, we achieve a **93.92% validation accuracy** in a single epoch.

In [ ]:
learner.fit_onecycle(2e-5, 1)



begin training using onecycle policy with max lr of 2e-05...
 762/1334 [================>.............] - ETA: 4:43:02 - loss: 0.4279 - accuracy: 0.7896

In [ ]:
learner.model.save_weights("ktrain_model.h5")